# Tutorial 6: GAE & VGAE  
Graph AutoEncoders GAE &  
Variational Graph Autoencoders VGAE    

[paper](https://arxiv.org/pdf/1611.07308.pdf)  
[code](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/autoencoder.py)

## Graph AutoEncoder GAE

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 2.9 MB/s 
     |████████████████████████████████| 3.5 MB 2.9 MB/s 


In [ ]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

### Load the data

In [ ]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Processing...
Done!


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [ ]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [ ]:
data = train_test_split_edges(data)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [ ]:
data

Data(x=[3327, 3703], y=[3327], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

### Define the Encoder

In [ ]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    

### Define the Autoencoder

In [ ]:
from torch_geometric.nn import GAE

In [ ]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [ ]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.6337, AP: 0.6519
Epoch: 002, AUC: 0.6552, AP: 0.6692
Epoch: 003, AUC: 0.6580, AP: 0.6722
Epoch: 004, AUC: 0.6616, AP: 0.6797
Epoch: 005, AUC: 0.6632, AP: 0.6863
Epoch: 006, AUC: 0.6626, AP: 0.6855
Epoch: 007, AUC: 0.6621, AP: 0.6845
Epoch: 008, AUC: 0.6620, AP: 0.6862
Epoch: 009, AUC: 0.6618, AP: 0.6885
Epoch: 010, AUC: 0.6615, AP: 0.6903
Epoch: 011, AUC: 0.6612, AP: 0.6919
Epoch: 012, AUC: 0.6610, AP: 0.6933
Epoch: 013, AUC: 0.6604, AP: 0.6951
Epoch: 014, AUC: 0.6600, AP: 0.6980
Epoch: 015, AUC: 0.6597, AP: 0.7008
Epoch: 016, AUC: 0.6591, AP: 0.7037
Epoch: 017, AUC: 0.6580, AP: 0.7055
Epoch: 018, AUC: 0.6575, AP: 0.7077
Epoch: 019, AUC: 0.6571, AP: 0.7103
Epoch: 020, AUC: 0.6569, AP: 0.7115
Epoch: 021, AUC: 0.6568, AP: 0.7126
Epoch: 022, AUC: 0.6572, AP: 0.7140
Epoch: 023, AUC: 0.6575, AP: 0.7156
Epoch: 024, AUC: 0.6576, AP: 0.7160
Epoch: 025, AUC: 0.6581, AP: 0.7170
Epoch: 026, AUC: 0.6589, AP: 0.7179
Epoch: 027, AUC: 0.6599, AP: 0.7190
Epoch: 028, AUC: 0.6614, AP:

In [ ]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[ 0.5298, -0.5418],
        [-1.0118,  1.0432],
        [ 0.7511, -0.7225],
        ...,
        [-0.4363,  0.4364],
        [ 0.7787, -0.7425],
        [ 0.7711, -0.7370]], grad_fn=<AddBackward0>)

### Are the results (AUC) and (AP) easy to read and compare?

## Use Tensorboard

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Import tensorboard

#### Installation: (if needed) "pip install tensorboard"

In [ ]:
writer = SummaryWriter('runs/GAE1_experiment_'+'2d_100_epochs')

In [ ]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6347, AP: 0.6360
Epoch: 002, AUC: 0.6439, AP: 0.6539
Epoch: 003, AUC: 0.6446, AP: 0.6555
Epoch: 004, AUC: 0.6477, AP: 0.6601
Epoch: 005, AUC: 0.6503, AP: 0.6644
Epoch: 006, AUC: 0.6523, AP: 0.6686
Epoch: 007, AUC: 0.6535, AP: 0.6723
Epoch: 008, AUC: 0.6542, AP: 0.6742
Epoch: 009, AUC: 0.6542, AP: 0.6751
Epoch: 010, AUC: 0.6543, AP: 0.6766
Epoch: 011, AUC: 0.6543, AP: 0.6787
Epoch: 012, AUC: 0.6549, AP: 0.6810
Epoch: 013, AUC: 0.6551, AP: 0.6834
Epoch: 014, AUC: 0.6551, AP: 0.6856
Epoch: 015, AUC: 0.6555, AP: 0.6895
Epoch: 016, AUC: 0.6555, AP: 0.6924
Epoch: 017, AUC: 0.6553, AP: 0.6956
Epoch: 018, AUC: 0.6555, AP: 0.6995
Epoch: 019, AUC: 0.6553, AP: 0.7023
Epoch: 020, AUC: 0.6547, AP: 0.7048
Epoch: 021, AUC: 0.6540, AP: 0.7062
Epoch: 022, AUC: 0.6533, AP: 0.7078
Epoch: 023, AUC: 0.6524, AP: 0.7085
Epoch: 024, AUC: 0.6517, AP: 0.7091
Epoch: 025, AUC: 0.6518, AP: 0.7101
Epoch: 026, AUC: 0.6521, AP: 0.7116
Epoch: 027, AUC: 0.6524, AP: 0.7128
Epoch: 028, AUC: 0.6533, AP:

### Graph Variational AutoEncoder (GVAE)

In [ ]:
from torch_geometric.nn import VGAE

In [ ]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)


class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [ ]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [ ]:
writer = SummaryWriter('runs/VGAE_experiment_'+'2d_100_epochs')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6136, AP: 0.6422
Epoch: 002, AUC: 0.6263, AP: 0.6626
Epoch: 003, AUC: 0.6308, AP: 0.6676
Epoch: 004, AUC: 0.6333, AP: 0.6699
Epoch: 005, AUC: 0.6336, AP: 0.6702
Epoch: 006, AUC: 0.6337, AP: 0.6702
Epoch: 007, AUC: 0.6334, AP: 0.6699
Epoch: 008, AUC: 0.6343, AP: 0.6703
Epoch: 009, AUC: 0.6359, AP: 0.6710
Epoch: 010, AUC: 0.6369, AP: 0.6710
Epoch: 011, AUC: 0.6374, AP: 0.6703
Epoch: 012, AUC: 0.6368, AP: 0.6687
Epoch: 013, AUC: 0.6346, AP: 0.6665
Epoch: 014, AUC: 0.6301, AP: 0.6624
Epoch: 015, AUC: 0.6231, AP: 0.6573
Epoch: 016, AUC: 0.6175, AP: 0.6534
Epoch: 017, AUC: 0.6119, AP: 0.6499
Epoch: 018, AUC: 0.6091, AP: 0.6483
Epoch: 019, AUC: 0.6122, AP: 0.6520
Epoch: 020, AUC: 0.6227, AP: 0.6605
Epoch: 021, AUC: 0.6325, AP: 0.6692
Epoch: 022, AUC: 0.6374, AP: 0.6754
Epoch: 023, AUC: 0.6403, AP: 0.6805
Epoch: 024, AUC: 0.6431, AP: 0.6848
Epoch: 025, AUC: 0.6446, AP: 0.6874
Epoch: 026, AUC: 0.6455, AP: 0.6884
Epoch: 027, AUC: 0.6463, AP: 0.6892
Epoch: 028, AUC: 0.6472, AP: